# Python网络爬虫初步

## 基础知识和准备

### 概念与基础知识

#### 超文本传输协议（HTTP）

超文本传输协议（HTTP）是一个用于传输超媒体文档（例如 HTML）的应用层协议。它是为 Web 浏览器与 Web 服务器之间的通信而设计的，但也可以用于其他目的。

HTTP 遵循经典的客户端-服务端模型，客户端打开一个连接以发出请求，然后等待它收到服务器端响应。HTTP 是无状态协议，这意味着服务器不会在两个请求之间保留任何数据（状态）。

**HTTP首部（HTTP Header）**

HTTP消息首部被用来描述资源信息，或是客户端和服务器的行为。

**HTTP请求方法**

可以使GET，POST方法来完成不同操作，同时也有一些其他的方法，如 OPTIONS，DELETE 和 TRACE。

**HTTP状态返回码**

HTTP响应状态代码指示特定HTTP请求是否已成功完成。例如200表示请求成功。

<br>

### Python相关库

#### [Requests库](https://requests.kennethreitz.org/en/master/)

Requests是一个优雅且简洁的HTTP库。

安装使用如下命令

```python
pip install requests
```

<br>

#### [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/index.html)

Beautiful Soup是用来从HTML或XML文件中提取数据的Python库。

安装使用如下命令

```python
pip install beautifulsoup4
```

Beautiful Soup支持Python标准库中的HTML解析器，还支持一些第三方的解析器，其中最常用的是lxml。安装lxml解析器，使用如下命令

```python
pip install lxml
```

## 编写网络爬虫

### 爬取第一个网页

- 使用requests库的get()函数

In [119]:
import requests

r = requests.get('http://www.baidu.com')

使用状态码查看是否爬取成功，如果返回200，就表示成功。

In [120]:
r.status_code

200

查看爬取的内容，使用text属性。

> 对于中文网页，如果是乱码，可以查看字符编码方式，并对此进行重新设置。

In [124]:
r.encoding = "UTF-8"
r.text

'<!DOCTYPE html>\r\n<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer><link rel=stylesheet type=text/css href=http://s1.bdstatic.com/r/www/cache/bdorz/baidu.min.css><title>百度一下，你就知道</title></head> <body link=#0000cc> <div id=wrapper> <div id=head> <div class=head_wrapper> <div class=s_form> <div class=s_form_wrapper> <div id=lg> <img hidefocus=true src=//www.baidu.com/img/bd_logo1.png width=270 height=129> </div> <form id=form name=f action=//www.baidu.com/s class=fm> <input type=hidden name=bdorz_come value=1> <input type=hidden name=ie value=utf-8> <input type=hidden name=f value=8> <input type=hidden name=rsv_bp value=1> <input type=hidden name=rsv_idx value=1> <input type=hidden name=tn value=baidu><span class="bg s_ipt_wr"><input id=kw name=wd class=s_ipt value maxlength=255 autocomplete=off autofocus></span><span class="bg s_btn_wr"><input type=submit id=su

### 网页解析初步

使用beautiful soup库进行HTML解析。



In [127]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.text, 'lxml')

In [15]:
print(soup.title)
print(soup.title.string)
print(soup.title.get_text())
print(soup.title.contents)

<title>百度一下，你就知道</title>
百度一下，你就知道
百度一下，你就知道
['百度一下，你就知道']


In [41]:
# to find links
soup.a

<a class="mnav" href="http://news.baidu.com" name="tj_trnews">新闻</a>

In [23]:
# to find all links

soup.find_all("a")

[<a class="mnav" href="http://news.baidu.com" name="tj_trnews">新闻</a>,
 <a class="mnav" href="http://www.hao123.com" name="tj_trhao123">hao123</a>,
 <a class="mnav" href="http://map.baidu.com" name="tj_trmap">地图</a>,
 <a class="mnav" href="http://v.baidu.com" name="tj_trvideo">视频</a>,
 <a class="mnav" href="http://tieba.baidu.com" name="tj_trtieba">贴吧</a>,
 <a class="lb" href="http://www.baidu.com/bdorz/login.gif?login&amp;tpl=mn&amp;u=http%3A%2F%2Fwww.baidu.com%2f%3fbdorz_come%3d1" name="tj_login">登录</a>,
 <a class="bri" href="//www.baidu.com/more/" name="tj_briicon" style="display: block;">更多产品</a>,
 <a href="http://home.baidu.com">关于百度</a>,
 <a href="http://ir.baidu.com">About Baidu</a>,
 <a href="http://www.baidu.com/duty/">使用百度前必读</a>,
 <a class="cp-feedback" href="http://jianyi.baidu.com/">意见反馈</a>]

#### CSS选择器与网页元素提取

使用.select()，它支持CSS选择器。

CSS选择器参考手册，点击[此处](https://www.w3school.com.cn/cssref/css_selectors.asp)。

In [137]:
# to extract a link
soup.select('[name="tj_trnews"]')[0].attrs["href"]

'http://news.baidu.com'

In [45]:
# to extract all links
for item in soup.find_all("a"):
    print(item.attrs["href"])

http://news.baidu.com
http://www.hao123.com
http://map.baidu.com
http://v.baidu.com
http://tieba.baidu.com
http://www.baidu.com/bdorz/login.gif?login&tpl=mn&u=http%3A%2F%2Fwww.baidu.com%2f%3fbdorz_come%3d1
//www.baidu.com/more/
http://home.baidu.com
http://ir.baidu.com
http://www.baidu.com/duty/
http://jianyi.baidu.com/


### 案例：高德地图

例如爬取上海的交通健康指数，点击[此处](https://report.amap.com/detail.do?city=310000)

In [138]:
import json

headers={"User-Agent" : "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9.1.6) ",
  "Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
  "Accept-Language" : "en-us",
  "Connection" : "keep-alive",
  "Accept-Charset" : "GB2312,utf-8;q=0.7,*;q=0.7"}

r = requests.get('https://report.amap.com/ajax/cityHourly.do?cityCode=110000&dataType=1', headers = headers)

r.json()

[[1573563600000, 1.28],
 [1573567200000, 1.22],
 [1573570800000, 1.15],
 [1573574400000, 1.12],
 [1573578000000, 1.09],
 [1573581600000, 1.08],
 [1573585200000, 1.08],
 [1573588800000, 1.06],
 [1573592400000, 1.08],
 [1573596000000, 1.24],
 [1573599600000, 1.76],
 [1573603200000, 1.93],
 [1573606800000, 1.71],
 [1573610400000, 1.55],
 [1573614000000, 1.38],
 [1573617600000, 1.31],
 [1573621200000, 1.37],
 [1573624800000, 1.46],
 [1573628400000, 1.46],
 [1573632000000, 1.57],
 [1573635600000, 2.03],
 [1573639200000, 2.07],
 [1573642800000, 1.56]]

In [139]:
import time

data = r.json()
for item in data:
    timearray = time.localtime(int(item[0]/1000))
    otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timearray)
    print(otherStyleTime, ": ", item[1])

2019-11-12 21:00:00 :  1.28
2019-11-12 22:00:00 :  1.22
2019-11-12 23:00:00 :  1.15
2019-11-13 00:00:00 :  1.12
2019-11-13 01:00:00 :  1.09
2019-11-13 02:00:00 :  1.08
2019-11-13 03:00:00 :  1.08
2019-11-13 04:00:00 :  1.06
2019-11-13 05:00:00 :  1.08
2019-11-13 06:00:00 :  1.24
2019-11-13 07:00:00 :  1.76
2019-11-13 08:00:00 :  1.93
2019-11-13 09:00:00 :  1.71
2019-11-13 10:00:00 :  1.55
2019-11-13 11:00:00 :  1.38
2019-11-13 12:00:00 :  1.31
2019-11-13 13:00:00 :  1.37
2019-11-13 14:00:00 :  1.46
2019-11-13 15:00:00 :  1.46
2019-11-13 16:00:00 :  1.57
2019-11-13 17:00:00 :  2.03
2019-11-13 18:00:00 :  2.07
2019-11-13 19:00:00 :  1.56
